# CS 584 :: Data Mining :: George Mason University :: Fall 2025


# Homework 3: Clustering

- **100 points [6% of your final grade]**
- **Due Sunday, November 16 by 11:59pm**

- *Goals of this homework:* (1) implement your K-means model; (2) implement K-means++; (3) Apply PCA to K-Means.

- *Submission instructions:* for this homework, you should submit your notebook file to **Canvas** (look for the homework 3 assignment there). Please name your submission **FirstName_Lastname_hw3.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw3.ipynb**. Your notebook should be fully executed so that we can see all outputs.

## Part 1: K-Means (60 points)

In this part, you will implement your own K-means algorithm to conduct clustering on handwritten digit images. In this homework, we will still use the handwritten digit image dataset we have already used in previous homework. However, since clustering is unsupervised learning, which means we do not use the label information for model training anymore. So, here, we will only use the testing data stored in the "test.txt" file.

First, let's load the data by executing the following code:

In [3]:
import numpy as np

test = np.loadtxt("/content/drive/MyDrive/hw3/test.txt", delimiter=',')
test_features = test[:, 1:]
test_labels = test[:, 0]
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing label matrix: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing label matrix: shape (10000,)


Now, it's time for you to implement your own K-means algorithm. First, please write your code to build your K-means model using the image data with **K = 10**, and **Euclidean distance**.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn**

**Note: you need to decide when to stop the iteration.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Write your code here


def pairwise_sq_dists(X, C):

    x2 = np.sum(X**2, axis=1, keepdims=True)
    c2 = np.sum(C**2, axis=1, keepdims=True).T
    xc = X @ C.T
    return x2 + c2 - 2*xc

def inertia(X, C, labels):
    D2 = pairwise_sq_dists(X, C)
    return np.sum(D2[np.arange(X.shape[0]), labels])

def kmeans(X, k=10, max_iters=100, tol=1e-4, random_state=0):
   
    rng = np.random.default_rng(random_state)
    n = X.shape[0]


    C = X[rng.choice(n, size=k, replace=False)].copy()
    labels = np.zeros(n, dtype=int)

    for it in range(1, max_iters + 1):

        D2 = pairwise_sq_dists(X, C)
        new_labels = np.argmin(D2, axis=1)

        assignments_unchanged = np.array_equal(new_labels, labels)
        labels = new_labels


        new_C = C.copy()
        for j in range(k):
            mask = (labels == j)
            if np.any(mask):
                new_C[j] = X[mask].mean(axis=0)
            else:

                new_C[j] = X[rng.integers(0, n)]


        shift = np.linalg.norm(new_C - C)
        C = new_C

        if assignments_unchanged or shift < tol:
            break

    return {"centroids": C, "labels": labels, "n_iter": it, "inertia": inertia(X, C, labels)}


result = kmeans(X, k=10, max_iters=100, tol=1e-4, random_state=0)
print(f"Iterations: {result['n_iter']}")
print(f"Inertia: {result['inertia']}")



Iterations: 94
Inertia: 25475281758.453735


Next, you need to calculate and print the **square root** of **S**um of **S**quared **E**rror (SSE) of each cluster generated by your K-means algorithm. Then, print out the averaged square root of SSE over all clusters.

**Note: The value of "square root of SSE" can diverge significantly. The expected value range is 10000~100000.**

In [6]:
# Write your code here



X = test[:, 1:]

centroids = result['centroids']
labels = result['labels']
k = 10

sqrt_sse_list = []

for j in range(k):
    cluster_points = X[labels == j]
    if len(cluster_points) == 0:
        sqrt_sse_list.append(0)
        continue


    sse_j = np.sum(np.sum((cluster_points - centroids[j])**2, axis=1))
    sqrt_sse_j = np.sqrt(sse_j)
    sqrt_sse_list.append(sqrt_sse_j)
    print(f"Cluster {j}: sqrt(SSE) = {sqrt_sse_j:.4f}")

avg_sqrt_sse = np.mean(sqrt_sse_list)
print(f"\nAverage sqrt(SSE) across all clusters: {avg_sqrt_sse:.4f}")


Cluster 0: sqrt(SSE) = 50706.9233
Cluster 1: sqrt(SSE) = 56729.2089
Cluster 2: sqrt(SSE) = 37498.9938
Cluster 3: sqrt(SSE) = 52162.2831
Cluster 4: sqrt(SSE) = 47007.1579
Cluster 5: sqrt(SSE) = 52316.5258
Cluster 6: sqrt(SSE) = 52049.5642
Cluster 7: sqrt(SSE) = 48918.7298
Cluster 8: sqrt(SSE) = 58764.6347
Cluster 9: sqrt(SSE) = 45350.1370

Average sqrt(SSE) across all clusters: 50150.4158


Then, please have a look on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure, and use this function to print out the homogeneity, completeness, and v-measure of your K-means model.

**Note: The values of homogeneity, completeness, and v-measure are expected to be >0.48**

In [7]:
# Write your code here


from sklearn.metrics import homogeneity_completeness_v_measure

true_labels = y.astype(int)
cluster_labels = result['labels'].astype(int)

h, c, v = homogeneity_completeness_v_measure(true_labels, cluster_labels)

print(f"Homogeneity:  {h:.4f}")
print(f"Completeness: {c:.4f}")
print(f"V-measure:    {v:.4f}")


Homogeneity:  0.5090
Completeness: 0.5159
V-measure:    0.5124


## Part 2: K-Means++ (30 points)

Ok, now you already have a good model. But can you further improve it? In the next cell, please implement the K-means++ model introduced in the lecture.

**Note: Everything else is the same as Part1, i.e., K=10, and you need to use Euclidean distance.**

In [ ]:
# Write your code here


import numpy as np
from sklearn.metrics import homogeneity_completeness_v_measure



def pairwise_sq_dists(X, C):
    x2 = np.sum(X**2, axis=1, keepdims=True)      
    c2 = np.sum(C**2, axis=1, keepdims=True).T    
    xc = X @ C.T                                   
    return x2 + c2 - 2*xc

def inertia(X, C, labels):
    D2 = pairwise_sq_dists(X, C)
    return np.sum(D2[np.arange(X.shape[0]), labels])

def init_kmeanspp(X, k, rng):
    
    n, d = X.shape
    C = np.empty((k, d), dtype=X.dtype)


    first = rng.integers(0, n)
    C[0] = X[first]


    d2 = pairwise_sq_dists(X, C[0:1]).reshape(-1)

    for i in range(1, k):
        probs = d2 / d2.sum()
        next_idx = rng.choice(n, p=probs)
        C[i] = X[next_idx]
        d2 = np.minimum(d2, pairwise_sq_dists(X, C[i:i+1]).reshape(-1))
    return C

def kmeans_kpp(X, k=10, max_iters=100, tol=1e-4, random_state=0):
    
    rng = np.random.default_rng(random_state)
    n = X.shape[0]


    C = init_kmeanspp(X, k, rng)
    labels = np.zeros(n, dtype=int)

    for it in range(1, max_iters + 1):

        D2 = pairwise_sq_dists(X, C)
        new_labels = np.argmin(D2, axis=1)
        assignments_unchanged = np.array_equal(new_labels, labels)
        labels = new_labels


        new_C = C.copy()
        for j in range(k):
            mask = (labels == j)
            if np.any(mask):
                new_C[j] = X[mask].mean(axis=0)
            else:
                new_C[j] = X[rng.integers(0, n)]


        shift = np.linalg.norm(new_C - C)
        C = new_C

        if assignments_unchanged or shift < tol:
            break

    return {"centroids": C, "labels": labels, "n_iter": it, "inertia": inertia(X, C, labels)}

res_kpp = kmeans_kpp(X, k=10, max_iters=100, tol=1e-4, random_state=0)
print(f"Iterations (K-means++): {res_kpp['n_iter']}")
print(f"Inertia    (K-means++): {res_kpp['inertia']:.4f}")


h, c, v = homogeneity_completeness_v_measure(y.astype(int), res_kpp['labels'].astype(int))
print(f"Homogeneity:  {h:.4f}")
print(f"Completeness: {c:.4f}")
print(f"V-measure:    {v:.4f}")

sqrt_sse = []
for j in range(10):
    pts = X[res_kpp['labels'] == j]
    if len(pts) == 0:
        val = 0.0
    else:
        sse_j = np.sum(np.sum((pts - res_kpp['centroids'][j])**2, axis=1))
        val = np.sqrt(sse_j)
    sqrt_sse.append(val)
    print(f"Cluster {j}: sqrt(SSE) = {val:.4f}")
print(f"\nAverage sqrt(SSE): {np.mean(sqrt_sse):.4f}")


Iterations (K-means++): 62
Inertia    (K-means++): 25368951813.1440
Homogeneity:  0.4936
Completeness: 0.5058
V-measure:    0.4996
Cluster 0: sqrt(SSE) = 62685.3835
Cluster 1: sqrt(SSE) = 39309.6142
Cluster 2: sqrt(SSE) = 56740.4748
Cluster 3: sqrt(SSE) = 49721.2929
Cluster 4: sqrt(SSE) = 40864.9993
Cluster 5: sqrt(SSE) = 36384.4404
Cluster 6: sqrt(SSE) = 48865.1298
Cluster 7: sqrt(SSE) = 59702.0378
Cluster 8: sqrt(SSE) = 40631.1715
Cluster 9: sqrt(SSE) = 60047.9666

Average sqrt(SSE): 49495.2511


In the next cell, use sklearn.metrics.homogeneity_completeness_v_measure() to print out the homogeneity, completeness, and v-measure of your K-means++ model.

In [9]:
# Write your code here



from sklearn.metrics import homogeneity_completeness_v_measure

true_labels = y.astype(int)
cluster_labels = res_kpp['labels'].astype(int)

h, c, v = homogeneity_completeness_v_measure(true_labels, cluster_labels)

print(f"Homogeneity (K-means++):  {h:.4f}")
print(f"Completeness (K-means++): {c:.4f}")
print(f"V-measure (K-means++):    {v:.4f}")


Homogeneity (K-means++):  0.4936
Completeness (K-means++): 0.5058
V-measure (K-means++):    0.4996


## Part 2: Dimension Reduction by PCA (10 points)

Last, in this part, please use PCA to reduce the feature dimension here. And then, apply your K-Means code to the reduced data and report homogeneity, completeness, and v-measure.

**Note: You need to consider the reduced dimension m of PCA as a hyper-parameter to tune. That is, you need to try different m and measure the corresponding clustering performance. At the end, you need to report the best m and clustering performance.**

**Note: Everything else is the same as Part1, i.e., K=10, and you need to use Euclidean distance.**

**Note: You can reuse your own code of PCA from your HW1, or you can directly use the PCA model from sklearn -- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html**

In [11]:

def pairwise_sq_dists(X, C):
    x2 = np.sum(X**2, axis=1, keepdims=True)
    c2 = np.sum(C**2, axis=1, keepdims=True).T
    xc = X @ C.T
    D2 = x2 + c2 - 2*xc

    return np.maximum(D2, 0.0)


def init_kmeanspp(X, k, rng):
    n, d = X.shape
    C = np.empty((k, d), dtype=X.dtype)


    first = rng.integers(0, n)
    C[0] = X[first]

    chosen = {first}

    d2 = pairwise_sq_dists(X, C[0:1]).reshape(-1)
    d2 = np.maximum(d2, 0.0)

    for i in range(1, k):
        s = d2.sum()

        if not np.isfinite(s) or s <= 0:

            candidates = np.array(list(set(range(n)) - chosen))
            if len(candidates) == 0:

                next_idx = rng.integers(0, n)
            else:
                next_idx = rng.choice(candidates)
        else:
            probs = d2 / s

            probs = np.clip(probs, 0.0, None)
            ps = probs.sum()
            if ps <= 0 or not np.isfinite(ps):
                candidates = np.array(list(set(range(n)) - chosen))
                next_idx = rng.choice(candidates) if len(candidates) else rng.integers(0, n)
            else:
                probs = probs / ps
                next_idx = rng.choice(n, p=probs)

        C[i] = X[next_idx]
        chosen.add(next_idx)

        newd2 = pairwise_sq_dists(X, C[i:i+1]).reshape(-1)
        d2 = np.minimum(d2, np.maximum(newd2, 0.0))

    return C


def kmeans_kpp(X, k=10, max_iters=100, tol=1e-4, random_state=0):
    rng = np.random.default_rng(random_state)
    n = X.shape[0]

    C = init_kmeanspp(X, k, rng)
    labels = np.zeros(n, dtype=int)

    for it in range(1, max_iters + 1):
        D2 = pairwise_sq_dists(X, C)
        new_labels = np.argmin(D2, axis=1)
        assignments_unchanged = np.array_equal(new_labels, labels)
        labels = new_labels

        new_C = C.copy()
        for j in range(k):
            mask = (labels == j)
            if np.any(mask):
                new_C[j] = X[mask].mean(axis=0)
            else:
                new_C[j] = X[rng.integers(0, n)]

        shift = np.linalg.norm(new_C - C)
        C = new_C
        if assignments_unchanged or shift < tol:
            break

    return {"centroids": C, "labels": labels, "n_iter": it, "inertia": np.sum(pairwise_sq_dists(X, C)[np.arange(n), labels])}


In [12]:
from sklearn.decomposition import PCA
from sklearn.metrics import homogeneity_completeness_v_measure

m_list = [10, 20, 30, 50, 80, 100, 150]
results = []

for m in m_list:
    pca = PCA(n_components=m, random_state=0)
    X_red = pca.fit_transform(X)

    res = kmeans_kpp(X_red, k=10, max_iters=100, tol=1e-4, random_state=0)
    h, c, v = homogeneity_completeness_v_measure(y.astype(int), res['labels'].astype(int))

    results.append((m, h, c, v, res['n_iter'], res['inertia']))
    print(f"m={m:>3} | iters={res['n_iter']:>3} | inertia={res['inertia']:.2f} | h={h:.4f} c={c:.4f} v={v:.4f}")

best = max(results, key=lambda t: t[3])
best_m, best_h, best_c, best_v, best_iters, best_inertia = best
print("\nBest result (by V-measure):")
print(f"m={best_m} | iters={best_iters} | inertia={best_inertia:.2f} | Homogeneity={best_h:.4f} | Completeness={best_c:.4f} | V-measure={best_v:.4f}")


m= 10 | iters= 47 | inertia=8562444622.39 | h=0.4960 c=0.5026 v=0.4993
m= 20 | iters=100 | inertia=13682328264.99 | h=0.5278 c=0.5338 v=0.5308
m= 30 | iters= 47 | inertia=16430696037.47 | h=0.5120 c=0.5193 v=0.5157
m= 50 | iters= 61 | inertia=19547219807.19 | h=0.4993 c=0.5030 v=0.5011
m= 80 | iters= 38 | inertia=21867310430.36 | h=0.5168 c=0.5234 v=0.5201
m=100 | iters= 87 | inertia=22519108609.14 | h=0.5058 c=0.5085 v=0.5071
m=150 | iters= 54 | inertia=23885455066.96 | h=0.4970 c=0.5224 v=0.5094

Best result (by V-measure):
m=20 | iters=100 | inertia=13682328264.99 | Homogeneity=0.5278 | Completeness=0.5338 | V-measure=0.5308


In the next cell, use sklearn.metrics.homogeneity_completeness_v_measure() to print out the homogeneity, completeness, and v-measure of your K-means model.

**Note: The values of homogeneity, completeness, and v-measure are expected to be >0.48**

In [13]:
# Write your code here

from sklearn.metrics import homogeneity_completeness_v_measure


from sklearn.decomposition import PCA
pca_final = PCA(n_components=20, random_state=0)
X_best = pca_final.fit_transform(X)


res_best = kmeans_kpp(X_best, k=10, max_iters=100, tol=1e-4, random_state=0)

h, c, v = homogeneity_completeness_v_measure(y.astype(int), res_best['labels'].astype(int))

print(f"Homogeneity (PCA + K-means):  {h:.4f}")
print(f"Completeness (PCA + K-means): {c:.4f}")
print(f"V-measure (PCA + K-means):    {v:.4f}")


Homogeneity (PCA + K-means):  0.5278
Completeness (PCA + K-means): 0.5338
V-measure (PCA + K-means):    0.5308
